To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [9]:
from pynq import Overlay
ol = Overlay("overlay/test_platform_kairos.bit")

To run only if you made change in the overlay :

In [10]:
ol.download()
# ol?

We define our SPI hardware object : AxiQspi.

In [11]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)
print(type(AxiQspi))

<class 'pynq.mmio.MMIO'>


Open the config file

In [12]:
with open("style.css","r") as file:
        css_file = file.read()
# print(css_file)

In [13]:
import yaml
with open("config.yaml","r") as file:
        config = yaml.safe_load(file)
        # print(config)

In [6]:
import yaml
with open("commands.yaml","r") as file:
        cmd_dict = yaml.safe_load(file)
        # print(config)

In [3]:
# %run nb/reports.ipynb
from nb import *

# Config setup
activate_exceptions = config["LAUNCH_CONFIG"]["activate_exceptions"]
max_while_iterations = config["LAUNCH_CONFIG"]["max_while_iterations"]

# Launch config : choose to run all tests (with exceptions) or run a single test : see config.yaml
single_test_config = config["LAUNCH_CONFIG"]["single_test"]
all_test_config = config["LAUNCH_CONFIG"]["all_tests"]
if (single_test_config["activate"] == all_test_config["activate"]):
    reports.test_config_error()

# Result file config : choose the result file extension to be html or txt : see config.yaml
activate_result_file_html = config["RESULT_FILE"]["html_config"]["activate"]
activate_result_file_txt = config["RESULT_FILE"]["txt_config"]["activate"]
activate_msg_prints = config["RESULT_FILE"]["print_msg"]["activate"]

ModuleNotFoundError: No module named 'builder'

In [1]:
# %run nb/parser.ipynb
# %run nb/package.ipynb
# %run nb/functions.ipynb
# %run nb/formater.ipynb
# %run nb/spi/spi_driver.ipynb
# %run nb/spi/spi_pkg.ipynb
# %run nb/reports.ipynb
# %run nb/translator.ipynb

from 
        

import os 

cnfg_master_spi(AxiQspi, clk_phase=0, clk_pol=0)

# ALL tests config :
if (all_test_config["activate"]) :
    all_test_files = os.listdir("./data")
    ignored_test_files = all_test_config['ignored_test_files']
    list_test_files = [f for f in all_test_files if f not in ignored_test_files]
    for test_file in list_test_files:
        test_file_name = test_file[:-4] # remove the txt extension
        test_file_path = 'data/' + test_file_name + '.txt'
        print(f"Currently sending test data from {test_file_name + '.txt'} ")
        
        if (activate_result_file_html):
            result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

        if (activate_result_file_txt):
            result_file_txt = init_txt_result_file(test_file_name, test_file_path)
        
        last_rd_data = [] # keep in memory the last read data for cmp commands
        with open(test_file_path,"r") as test_file_opened:
            parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
            number_cmd_test_file = len(parsed_file)
            for indice_command_from_test_file in range(1,number_cmd_test_file+1):
                parsed_line = parsed_file[indice_command_from_test_file]
                last_rd_data = command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
                
        if (activate_result_file_html):
            end_html_result_file()

        if (activate_result_file_txt):
            end_txt_result_file()
                
        print(f"Finished sending test data from {test_file + '.txt'} ")
     
    
    
# SINGLE test config :
if (single_test_config["activate"]) :
    test_file_name = single_test_config['test_file'] 
    test_file_path = 'data/' + test_file_name + '.txt'
    print(f"Currently sending test data from {test_file_name + '.txt'} ")

    if (activate_result_file_html):
        result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

    if (activate_result_file_txt):
        result_file_txt = init_txt_result_file(test_file_name, test_file_path)
    
    last_rd_data = [] # keep in memory th  e last read data for cmp commands
    with open(test_file_path,"r") as test_file_opened: 
        parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
        number_cmd_test_file = len(parsed_file)
        for indice_command_from_test_file in range(1,number_cmd_test_file+1):
            parsed_line = parsed_file[indice_command_from_test_file]
            last_rd_data = command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
            
    if (activate_result_file_html):
        end_html_result_file()

    if (activate_result_file_txt):
        end_txt_result_file()
            
    print(f"Finished sending test data from {test_file_name + '.txt'} ")
    

NameError: name 'config' is not defined

NameError: name 'config' is not defined

In [ ]:
# Show clocks frequency
# from pynq import Clocks
# print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
# print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')